# Título   Crear clase sample

## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Crear un sample del dataset con proporciones parametrizadas

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [1]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

feature_a_usar = 'clase_ternaria'     #feature a la que hacen referencia los valores de % a seleccionar
sample_pct={'BAJA+1':1, 'BAJA+2':1, 'CONTINUA':.05}   # % de cada clase a obtener en el sample

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [11]:
# Ejemplo


dataset_path = '~/buckets/b1/datasets' 
dataset_file = 'competencia_02.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [4]:
output_file='competencia_02_sampleado.csv.gz'


In [5]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [6]:
import os
import pandas as pd

In [7]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [12]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          249221109    201901               1            0         1   
1          249221468    201901               1            0         1   
2          249223005    201901               1            0         1   
3          249228180    201901               1            0         1   
4          249232117    201901               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            59                 276        7597.55              47433.58   
1            51                  13         738.16               -325.09   
2            46                 178        1014.31               9434.15   
3            64                 296        1028.33               6873.80   
4            77                 349       11617.61              43291.55   

   mcomisiones  ...  Visa_madelantodolares  Visa_fultimo_cierre  Visa_mpagado  \
0      5654.59  ...    

In [13]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [14]:
df["cli_mes"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].astype(str)
df["cli_mesMas1"]=df["numero_de_cliente"].astype(str)+(df["foto_mes"]+1).astype(str)
df["cli_mesMas2"]=df["numero_de_cliente"].astype(str)+(df["foto_mes"]+2).astype(str)


In [15]:
df1=df['cli_mes']   #sólo las columnas del cliente
# Realiza la unión entre df y df1
resultado = df.merge(df1, left_on='cli_mesMas1', right_on='cli_mes', how='left', indicator=True)
df['existe1'] = resultado['_merge'] == 'both'
resultado = df.merge(df1, left_on='cli_mesMas2', right_on='cli_mes', how='left', indicator=True)
df['existe2'] = resultado['_merge'] == 'both'


In [16]:
df[ "clase_ternaria"]="CONTINUA"          #default
df.loc[df["existe2"]==False, "clase_ternaria"]="BAJA+2"
df.loc[df["existe1"]==False, "clase_ternaria"]="BAJA+1"
df.loc[df["existe2"]==True, "clase_ternaria"]="CONTINUA"

In [17]:
# Eliminar los campos creados
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])

In [ ]:
ruta_nuevo_archivo = os.path.join(dataset_path, output_file)
# Guarda el DataFrame en el nuevo archivo
df.to_csv(ruta_nuevo_archivo, index=False, compression='gzip')
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")